In [46]:
# imports
import pandas as pd
import os # use this to access your environment variables
import requests # this will be used to call the APIs

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [48]:
FOURSQUARE_KEY = os.getenv('FOURSQUARE_API_KEY')
YELP_KEY = os.getenv('YELP_API_KEY')
print(FOURSQUARE_KEY)
print(YELP_KEY)
# confirm both API keys have been imported successfully from Terminal

fsq3F074rL3S6i0kGeOeRWB1KhL1WLyqB4dyTOG72DKFm4A=
Crgar6c-4CFZps62GfA4OGaN1ClkE5OfzQoHOVBrC3njmSxaimHL02E2srHd7-WnFVSrXuD4xQpXuzn6KwIS9MwTqT67Xxwm4ddGwHES6kS7aTEWrutBhqA4KHSuZnYx


In [50]:
def get_venues_fs(latitude, longitude, radius, api_key, categories, limit):
    """
    Get amenities and POIs from Foursquare API call
    Args:
        latitude (float): latitude for query (must be combined with longitude)
        longitude (float): longitude for query (must be combined with latitude)
        api_key (str): foursquare API to use for query (imported in line above)
        categories (str) : Foursquare-recognized place type. If not passed no place_type will be specified. Separate ids with commas
    
    Returns:
        response: response object from the requests library.
    """
    url = "https://api.foursquare.com/v3/places/search"
    
    headers = {
        "Accept": "application/json",
        "Authorization": api_key
    }
    
    params = {
        "ll": f"{latitude},{longitude}",
        "radius": radius,
        "categories": categories,
        "limit": limit
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200: # 200 is success
        return response.json()
    else:
        response.raise_for_status()
        

In [56]:
# testing

categories = '10035,13003,13065,16000' # See following cell bullet points for category names - bars, restaurants, live shows, outdoors

res = get_venues_fs(latitude=43.664467, longitude=-79.414783, radius=750, api_key=FOURSQUARE_KEY, categories=categories, limit=50)

print(len(res['results']))
print("\n")
print(res['results'][3]) # main body of a specific venue result
print("\n")

print(res['results'][3]['name'])  # this is how you access the name, we could loop through to get names

print(res['results'][3]['location']['address']) # accessing the address for the dataframe

print(res['results'][3]['categories'][0]['name']) # accessing the first general category of the venue, most only have one
print("\n")

for venue in res['results']:
    name = venue['name']
    address = venue['location'].get('address', 'Address not available')
    first_category = venue['categories'][0]['name']
    category_names = [category['name'] for category in venue['categories']] # there are multiple categories for some venues
    print(f"{name}, {address}, {first_category}, {category_names}")

    
# print(res['results'])

44


{'fsq_id': '4b11b1f2f964a520778223e3', 'categories': [{'id': 13026, 'name': 'BBQ Joint', 'short_name': 'BBQ', 'plural_name': 'BBQ Joints', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bbqalt_', 'suffix': '.png'}}, {'id': 13289, 'name': 'Korean Restaurant', 'short_name': 'Korean', 'plural_name': 'Korean Restaurants', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/korean_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'LikelyOpen', 'distance': 24, 'geocodes': {'main': {'latitude': 43.664553, 'longitude': -79.414501}, 'roof': {'latitude': 43.664553, 'longitude': -79.414501}}, 'link': '/v3/places/4b11b1f2f964a520778223e3', 'location': {'address': '628 Bloor St W', 'country': 'CA', 'formatted_address': '628 Bloor St W, Toronto ON M6G 1K7', 'locality': 'Toronto', 'postcode': 'M6G 1K7', 'region': 'ON'}, 'name': 'Korean Village Restaurant', 'related_places': {}, 'timezone': 'America/Toronto'}


Korean Village Restaurant
628 Bloor St W
BBQ Joint



**Notes on the test API Call and justification of parameters:**
- Downtown Toronto is very densely packed with bike stations and the amenities I will be searching for. The default radius is 5km which is too large to capture the relationship between bike/dock availability and amenities. The appropriate range would be either **1km or even 750m**
- Foursquare Category IDs of interest are: 10035 (Performing Arts Venue), 12013 (College and University amenities), 13003 (Bars), 13065 (Restaurants)
- Other candidates: 16000 (landmarks & outdoors), 17057 (Food & Beverage Retail), 18000 (Sports & Recreaction)

**Notes on the information to return in dataframe**
- Name of Venue
- Address
- Venue Categories
- The latitude and longitude used to call the results belongs to the bike dock

The unified dataframe is about the bike docks/stations and this data will return number of bars, number of restaurants, performing arts venues, outdoor venues

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Yelp

**Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice.**

My notes on Yelp Fusion API (https://docs.developer.yelp.com/docs/fusion-intro):
- They have a number of different types of search including search for events, search by business ID, others... see https://docs.developer.yelp.com/docs/resources-categories
- To compare closest with Foursquare we will use the standard businesses search https://docs.developer.yelp.com/reference/v3_business_search
- The starter API package offers review count and average review star rating
- First 300 calls are free and it's 7.99 for 1000 API calls
- We would need to make at least 820 api calls for each of the bike stations

In [30]:
def get_venues_yelp(latitude, longitude, radius, yelp_key, categories, limit):
    """
    Get amenities and POIs from Yelp API call
    Args:
        latitude (float): latitude for query (must be combined with longitude)
        longitude (float): longitude for query (must be combined with latitude)
        yelp_key (str): yelp API to use for query (see imports in first cell)
        categories (str) : Place type, wit Yelp API they are descriptive eg. "bar,restaurant" unlike Foursquare's 5-digit codes. Separate ids with commas
    Returns:
        response: response object from the requests library.
    """
    url = "https://api.yelp.com/v3/businesses/search"
    
    headers = {
        "Accept": "application/json",
        "Authorization": yelp_key
    }
    
    params = {
        "latitude": f"{latitude}",
        "longitude": f"{longitude}",
        "radius": radius,
        "categories": categories,
        "limit": limit
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200: # 200 is success
        return response.json()
    else:
        response.raise_for_status()   

In [38]:
# testing

yelp_categories = 'cafes,bars,restaurants,parks,musicvenues' # bars, restaurants, live shows, outdoors

yelp_res = get_venues_yelp(latitude=43.664467, longitude=-79.414783, radius=750, yelp_key=YELP_KEY, categories=yelp_categories, limit=50)

print(yelp_res)

HTTPError: 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?latitude=43.664467&longitude=-79.414783&radius=750&categories=cafes%2Cbars%2Crestaurants%2Cparks%2Cmusicvenues&limit=50

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating